#DATA COLLECTION

In [2]:
!rm -rf ~/.kaggle
!pip install -q kaggle

In [3]:
!pwd

/content


In [4]:
import json
token = {"username":"manjuladevis20ada30","key":"1268523272c377ab2cf0d299cfef7a78"}

In [5]:
with open("kaggle.json", "w") as file:
  json.dump(token, file)

In [6]:
!mkdir ~/.kaggle

In [7]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [8]:
!chmod 600 ~/.kaggle/kaggle.json

In [9]:
!kaggle datasets download -d manjusundarraj/garbage

 54% 22.0M/40.9M [00:00<00:00, 103MB/s] 
100% 40.9M/40.9M [00:00<00:00, 136MB/s]


In [10]:
!unzip /content/garbage.zip -d /content/garbageclassification

Archive:  /content/garbage.zip
  inflating: /content/garbageclassification/Garbage classification/testing/cardboard/cardboard1.jpg  
  inflating: /content/garbageclassification/Garbage classification/testing/cardboard/cardboard10.jpg  
  inflating: /content/garbageclassification/Garbage classification/testing/cardboard/cardboard11.jpg  
  inflating: /content/garbageclassification/Garbage classification/testing/cardboard/cardboard12.jpg  
  inflating: /content/garbageclassification/Garbage classification/testing/cardboard/cardboard13.jpg  
  inflating: /content/garbageclassification/Garbage classification/testing/cardboard/cardboard14.jpg  
  inflating: /content/garbageclassification/Garbage classification/testing/cardboard/cardboard15.jpg  
  inflating: /content/garbageclassification/Garbage classification/testing/cardboard/cardboard16.jpg  
  inflating: /content/garbageclassification/Garbage classification/testing/cardboard/cardboard17.jpg  
  inflating: /content/garbageclassification

#IMAGE PREPROCESSING

In [12]:
# Importing the libraries
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
!ls

cb.jfif  garbageclassification	garbage.zip  kaggle.json  sample_data


In [14]:
# Configure ImageDataGenerator class
train_data_dir = '/content/garbageclassification/Garbage classification/training'
test_data_dir = '/content/garbageclassification/Garbage classification/testing'

img_width, img_height = 150, 150
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [15]:
# Apply ImageDataGenerator functionality to train set and test set
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 2407 images belonging to 6 classes.
Found 120 images belonging to 6 classes.


## MODEL BUILDING

In [16]:
# Importing the model building libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

In [17]:
# Initialize the model
model = Sequential()

In [18]:
# Adding CNN layers
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

In [19]:
# Adding Dense layers
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6))  # Number of classes
model.add(Activation('softmax'))

In [20]:
#summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 activation (Activation)     (None, 148, 148, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 72, 72, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                    

In [21]:
# Configure the learning process
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [22]:
# Training the model
epochs = 10
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

<ipython-input-22-42bfeebab4e0>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
75/75 [==============================] - 123s 2s/step - loss: 1.7911 - accuracy: 0.2312 - val_loss: 1.6809 - val_accuracy: 0.2604
Epoch 2/10
75/75 [==============================] - 106s 1s/step - loss: 1.6026 - accuracy: 0.3179 - val_loss: 1.4973 - val_accuracy: 0.3542
Epoch 3/10
75/75 [==============================] - 103s 1s/step - loss: 1.5254 - accuracy: 0.3579 - val_loss: 1.4343 - val_accuracy: 0.5000
Epoch 4/10
75/75 [==============================] - 101s 1s/step - loss: 1.4551 - accuracy: 0.4029 - val_loss: 1.6541 - val_accuracy: 0.3646
Epoch 5/10
75/75 [==============================] - 101s 1s/step - loss: 1.4062 - accuracy: 0.4446 - val_loss: 1.3123 - val_accuracy: 0.4688
Epoch 6/10
75/75 [==============================] - 104s 1s/step - loss: 1.3614 - accuracy: 0.4446 - val_loss: 1.3894 - val_accuracy: 0.3542
Epoch 7/10
75/75 [==============================] - 108s 1s/step - loss: 1.3440 - accuracy: 0.4745 - val_loss: 1.3098 - val_accuracy: 0.4167
Epoch 8/10
75

In [23]:
#testing the model
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate_generator(validation_generator, validation_generator.samples // batch_size)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)


<ipython-input-23-d6009a3c9f26>:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy = model.evaluate_generator(validation_generator, validation_generator.samples // batch_size)


Test Accuracy: 0.375


In [24]:
from tensorflow.keras.preprocessing import image

# Load the sample image
sample_image_path = "/content/cb.jfif"
sample_image = image.load_img(sample_image_path, target_size=(150, 150))
sample_image = image.img_to_array(sample_image)
sample_image = np.expand_dims(sample_image, axis=0)
sample_image /= 255.0
a=np.argmax(model.predict(sample_image),axis=1)
index=['0','1','2','3','4','5']
result=str(index[a[0]])
result

1/1 [==============================] - 0s 188ms/step


'3'

In [24]:
train_generator.class_indices

In [27]:
index1 = ['cardboard','glass','metal','paper','plastic','trash']
result1=str(index1[a[0]])
result1

'paper'

## SAVING THE MODEL

In [28]:
# Step 4: Save the model
model.save('garbage_classification_model.h5')
print("Model saved successfully!")

Model saved successfully!
